In [12]:
import numpy as np
import pandas as pd
import sklearn
import scipy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier

# Define Class

In [13]:
class DataframeFunctionTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self

### DataFrame Transformer

In [14]:
def process_dataframe(input_df):
    input_df.drop('sex', axis = 1, inplace = True)
    return input_df

# sample dataframe
df = pd.DataFrame({
    "id":[1,2,3,4],
    "text":["foo","Bar","BAz","quux"],
    "sex": [1,0,1,1]
})

#process_dataframe(df)

In [15]:
# this pipeline has a single step
pipeline = Pipeline([
    ("uppercase", DataframeFunctionTransformer(process_dataframe))
])

# apply the pipeline to the input dataframe
pipeline.fit_transform(df)

,id,text
0,1,foo
1,2,Bar
2,3,BAz
3,4,quux


### To Dense 

In [16]:
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

class ToDenseTransformer():

    # here you define the operation it should perform
    def transform(self, X, y=None, **fit_params):
        return X.todense()

    # just return self
    def fit(self, X, y=None, **fit_params):
        return self

In [17]:
data = scipy.sparse.csr_matrix([
    [1.,0.,0.,0.,0.,0.],
    [0.,1.,0.,0.,0.,0.],
    [1.,0.,0.,0.,0.,0.],
    [0.,0.,0.,0.,1.,0.],
    [0.,0.,0.,1.,0.,0.],
    [1.,0.,0.,0.,0.,0.],
    [1.,1.,0.,0.,0.,0.],
    [1.,1.,0.,0.,0.,0.],
])

target = np.array([1,1,1,0,0,0,1,1])

In [18]:
# need to make matrices dense because PCA does not work with sparse vectors.
pipeline = Pipeline([
    ('to_dense',ToDenseTransformer()),
    ('pca',PCA()),
    ('clf',DecisionTreeClassifier())
])

pipeline.fit(data,target)
pipeline.predict(data)

array([1, 1, 1, 0, 0, 1, 1, 1])

### Missing Imputation

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [20]:
df = pd.DataFrame({
    'name':['alice','bob','charlie','david','edward'],
    'age':[24,32,np.nan,38,20]
})

df.head()

,name,age
0,alice,24.0
1,bob,32.0
2,charlie,NaN
3,david,38.0
4,edward,20.0


In [24]:
transformer_step = ColumnTransformer([('impute_mean', SimpleImputer(strategy='mean'), ['age'])], remainder='passthrough')

In [25]:
pipe = Pipeline([
    ('transformer', transformer_step)
])

In [26]:
pipe.fit(df)

pd.DataFrame(
    data=pipe.transform(df),
    columns=['age', 'name']
)[["name","age"]]

,name,age
0,alice,24
1,bob,32
2,charlie,28.5
3,david,38
4,edward,20


### ColumnTransformer with OneHotEncoder

In [67]:
from sklearn.preprocessing import OneHotEncoder

In [68]:
df = pd.DataFrame({
    'favorite_color':['blue','green','red','green','blue'],
    'age': [10,15,10,np.nan,10],
    'target':[1,0,1,0,1]
})
df

,favorite_color,age,target
0,blue,10.0,1
1,green,15.0,0
2,red,10.0,1
3,green,NaN,0
4,blue,10.0,1


In [69]:
# define individual transformers in a pipeline
categorical_preprocessing = Pipeline([('ohe', OneHotEncoder())])
numerical_preprocessing = Pipeline([('imputation', SimpleImputer())])

# define which transformer applies to which columns
preprocess = ColumnTransformer([
    ('categorical_preprocessing', categorical_preprocessing, ['favorite_color']),
    ('numerical_preprocessing', numerical_preprocessing, ['age'])
])

# create the final pipeline with preprocessing steps and 
# the final classifier step
pipeline = Pipeline([
    ('preprocess', preprocess),
    ('clf', DecisionTreeClassifier())
])

# now fit the pipeline using the whole dataframe
df_features = df[['favorite_color','age']]
df_target = df['target']

pipeline.fit(df_features, df_target)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('categorical_preprocessing',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder())]),
                                                  ['favorite_color']),
                                                 ('numerical_preprocessing',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer())]),
                                                  ['age'])])),
                ('clf', DecisionTreeClassifier())])

In [28]:
pipeline.predict(df_features)

array([1, 0, 1, 0, 1])

### Select columns with Custom Transformer

In [29]:
from sklearn.base import TransformerMixin,BaseEstimator

In [30]:
class SelectColumnsTransfomer():
    def __init__(self, columns=None):
        self.columns = columns

    def transform(self, X, **transform_params):
        cpy_df = X[self.columns].copy()
        return cpy_df

    def fit(self, X, y=None, **fit_params):
        return self

In [31]:
df = pd.DataFrame({
    'name':['alice','bob','charlie','david','edward'],
    'age':[24,32,np.nan,38,20]
})

df.head()

,name,age
0,alice,24.0
1,bob,32.0
2,charlie,NaN
3,david,38.0
4,edward,20.0


In [32]:
pipe = Pipeline([
    ('selector', SelectColumnsTransfomer(["name"]))
])

In [37]:
pipe.fit_transform(df)

,name
0,alice
1,bob
2,charlie
3,david
4,edward


In [38]:
pipe.transform(df)

,name
0,alice
1,bob
2,charlie
3,david
4,edward


### Function Transformer with Parameters

In [44]:
from nltk.stem import RSLPStemmer
import nltk
nltk.download('rslp')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer

[nltk_data] Downloading package rslp to /home/david/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [45]:
def stem_str(input_series, stemmer):
    
    def stem(input_str):
        return " ".join([stemmer.stem(t) for t in input_str.split(" ")]).strip()
    
    return input_series.apply(stem)

In [46]:
pipeline = Pipeline([
    ('stemmer', FunctionTransformer(
        func=stem_str, 
        kw_args={'stemmer': RSLPStemmer()})),
    ('vect', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

In [48]:
df = pd.DataFrame({
    'text':[
        'Lorem ipsum dolor sit amet, consectetur adipiscing elit.',
        'Sed accumsan congue enim non pretium.',
        'In hac habitasse platea dictumst.',
        'Sed tincidunt ipsum nec urna vulputate luctus.'
    ],
    'target':[0, 1, 0, 1]
})
df

,text,target
0,"Lorem ipsum dolor sit amet, consectetur adipis...",0
1,Sed accumsan congue enim non pretium.,1
2,In hac habitasse platea dictumst.,0
3,Sed tincidunt ipsum nec urna vulputate luctus.,1


In [49]:
pipeline.fit(df['text'],df['target'])

Pipeline(steps=[('stemmer',
                 FunctionTransformer(func=<function stem_str at 0x7f33e8b59820>,
                                     kw_args={'stemmer': <nltk.stem.rslp.RSLPStemmer object at 0x7f33e8b12550>})),
                ('vect', TfidfVectorizer()), ('clf', LogisticRegression())])

In [50]:
pipeline.predict(df['text'])

array([0, 1, 0, 1])